# [Module 1.6] 피쳐 엔지니어링

이 노트북은 아래와 같은 피쳐 엔지니어링을 통하여 새로운 피쳐를 생성 합니다.
- 날짜관련 피쳐 생성(월, 일, 요일)
- 기존의 피쳐들을 결합하여 새로운 피쳐 생성 (피쳐1 + 피쳐2 = 뉴피쳐)
    - 아래는 customer와 seller 가 가까운 거리에 있을때에 배송 시간이 짧을 것 같다는 생각으로 새로운 피쳐를 생성 합니다.
        - customer_seller_state
        - customer_seller_city
        - customer_seller_zipcode
    - 아래는 product의 가로, 세로, 높이는 각각이 의미가 적으므로, 세 개를 곱하여 볼륨이라는 피쳐를 생성 합니다.
        - product_volume 
- Train, Test 데이터 세트로 분리
    - **현재의 문제는 시간의 경과에 따라서, 훈련/검증을 분리를 하는 것이 맞기에 시간의 경과에 따라 분리 합니다. 또한 실제 서비스를 할 때에도, 시간의 경과에 따라 추론을 하기에 이와 같이 분리 합니다.**
    - 시간순으로 정렬 후에 8:2 로 분리
- AutoGluo에 사용할 최종 컬럼 선택
- 로컬에 파일 저장

In [1]:
import pandas as pd
pd.options.display.max_rows=5
import numpy as np

In [2]:
%store -r full_data_file_name

### 데이터 로딩 및 셔플링

In [3]:
df = pd.read_csv(full_data_file_name)
df = df.sample(frac=1.0, random_state=1000)
df

,classes,order_approved_at,customer_id,customer_zip_code_prefix,customer_city,customer_state,price,freight_value,product_id,product_weight_g,product_length_cm,product_height_cm,product_width_cm,product_category_name_english,seller_zip_code_prefix,seller_city,seller_state
37413,3,2018-07-14 13:04:07,4ee61c3905a5c398d44b089108961bb3,28950,armacao dos buzios,RJ,105.00,27.04,2f13d1dc8b4e1d9d8027be50339546a9,2650.0,30.0,30.0,30.0,furniture_decor,3204,sao paulo,SP
54762,3,2017-03-25 10:25:16,959292edcade77d6b60dc8f49f01cd71,37880,cabo verde,MG,23.99,14.52,b000447e24e31a4d7e628ca4d0622131,250.0,19.0,4.0,11.0,telephony,3504,sao paulo,SP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18782,3,2018-07-16 18:20:34,609b34a18fd0d61719d0b3190ec05231,21240,rio de janeiro,RJ,200.00,19.50,92d5ae8e42c4599266f210bf0469ae9b,439.0,18.0,11.0,13.0,watches_gifts,14050,ribeirao preto,SP
3776,1,2018-05-25 07:06:30,dc275c5e585b7c3a3cddef527e34fc19,26540,nilopolis,RJ,18.90,7.55,15de022edf1005363381e66bed514528,100.0,16.0,3.0,23.0,furniture_decor,20270,rio de janeiro,RJ


In [4]:
df.columns

Index(['classes', 'order_approved_at', 'customer_id',
       'customer_zip_code_prefix', 'customer_city', 'customer_state', 'price',
       'freight_value', 'product_id', 'product_weight_g', 'product_length_cm',
       'product_height_cm', 'product_width_cm',
       'product_category_name_english', 'seller_zip_code_prefix',
       'seller_city', 'seller_state'],
      dtype='object')

## 날짜 피쳐 생성: Month, Day, WeeoOfDay(요일)

In [5]:
def create_date_feature(raw_df):
    df = raw_df.copy()
    df['order_date'] = pd.to_datetime(df['order_approved_at'])    
    df['order_weekday'] = df['order_date'].dt.weekday
    df['order_day'] = df['order_date'].dt.day    
    df['order_month'] = df['order_date'].dt.month        
    return df

f_df = create_date_feature(df)
f_df

,classes,order_approved_at,customer_id,customer_zip_code_prefix,customer_city,customer_state,price,freight_value,product_id,product_weight_g,...,product_height_cm,product_width_cm,product_category_name_english,seller_zip_code_prefix,seller_city,seller_state,order_date,order_weekday,order_day,order_month
37413,3,2018-07-14 13:04:07,4ee61c3905a5c398d44b089108961bb3,28950,armacao dos buzios,RJ,105.00,27.04,2f13d1dc8b4e1d9d8027be50339546a9,2650.0,...,30.0,30.0,furniture_decor,3204,sao paulo,SP,2018-07-14 13:04:07,5,14,7
54762,3,2017-03-25 10:25:16,959292edcade77d6b60dc8f49f01cd71,37880,cabo verde,MG,23.99,14.52,b000447e24e31a4d7e628ca4d0622131,250.0,...,4.0,11.0,telephony,3504,sao paulo,SP,2017-03-25 10:25:16,5,25,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18782,3,2018-07-16 18:20:34,609b34a18fd0d61719d0b3190ec05231,21240,rio de janeiro,RJ,200.00,19.50,92d5ae8e42c4599266f210bf0469ae9b,439.0,...,11.0,13.0,watches_gifts,14050,ribeirao preto,SP,2018-07-16 18:20:34,0,16,7
3776,1,2018-05-25 07:06:30,dc275c5e585b7c3a3cddef527e34fc19,26540,nilopolis,RJ,18.90,7.55,15de022edf1005363381e66bed514528,100.0,...,3.0,23.0,furniture_decor,20270,rio de janeiro,RJ,2018-05-25 07:06:30,4,25,5


## 기존 피쳐 결합하여 새로운 피쳐 생성 (컬럼1 + 컬럼2 = 뉴피쳐)

In [6]:
def change_var_type(f_df):
    df = f_df.copy()
    df['customer_zip_code_prefix'] = df['customer_zip_code_prefix'].astype(str)
    df['seller_zip_code_prefix'] = df['seller_zip_code_prefix'].astype(str)    
    return df

def comnbine_columns(f_df,src_col1, src_col2,new_col):
    df = f_df.copy()
    df[new_col] = df[str(src_col1)] + '_' + df[str(src_col2)]
    print("df shape: ", df.shape)
    return df



f_df = change_var_type(f_df)

### custoemr_state + seller_state

In [7]:
f_df = comnbine_columns(f_df,src_col1='customer_state', src_col2='seller_state',new_col='customer_seller_state')

df shape:  (67176, 22)


### custoemr_city + seller_city

In [8]:
f_df = comnbine_columns(f_df,src_col1='customer_city', src_col2='seller_city',new_col='customer_seller_city')

df shape:  (67176, 23)


### custoemr_zip + seller_zip

In [9]:
f_df = comnbine_columns(f_df,src_col1='customer_zip_code_prefix', 
                        src_col2='seller_zip_code_prefix',new_col='customer_seller_zip_code_prefix')

df shape:  (67176, 24)


In [10]:
f_df

,classes,order_approved_at,customer_id,customer_zip_code_prefix,customer_city,customer_state,price,freight_value,product_id,product_weight_g,...,seller_zip_code_prefix,seller_city,seller_state,order_date,order_weekday,order_day,order_month,customer_seller_state,customer_seller_city,customer_seller_zip_code_prefix
37413,3,2018-07-14 13:04:07,4ee61c3905a5c398d44b089108961bb3,28950,armacao dos buzios,RJ,105.00,27.04,2f13d1dc8b4e1d9d8027be50339546a9,2650.0,...,3204,sao paulo,SP,2018-07-14 13:04:07,5,14,7,RJ_SP,armacao dos buzios_sao paulo,28950_3204
54762,3,2017-03-25 10:25:16,959292edcade77d6b60dc8f49f01cd71,37880,cabo verde,MG,23.99,14.52,b000447e24e31a4d7e628ca4d0622131,250.0,...,3504,sao paulo,SP,2017-03-25 10:25:16,5,25,3,MG_SP,cabo verde_sao paulo,37880_3504
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18782,3,2018-07-16 18:20:34,609b34a18fd0d61719d0b3190ec05231,21240,rio de janeiro,RJ,200.00,19.50,92d5ae8e42c4599266f210bf0469ae9b,439.0,...,14050,ribeirao preto,SP,2018-07-16 18:20:34,0,16,7,RJ_SP,rio de janeiro_ribeirao preto,21240_14050
3776,1,2018-05-25 07:06:30,dc275c5e585b7c3a3cddef527e34fc19,26540,nilopolis,RJ,18.90,7.55,15de022edf1005363381e66bed514528,100.0,...,20270,rio de janeiro,RJ,2018-05-25 07:06:30,4,25,5,RJ_RJ,nilopolis_rio de janeiro,26540_20270


## Product volume 컬럼 생성 (가로 * 세로 * 높이 의 계산값)

In [11]:
def add_product_volume(raw_df):
    df = raw_df.copy()
    df['product_volume'] = df.product_length_cm * df.product_width_cm * df.product_height_cm
    return df

f_df = add_product_volume(f_df)

In [12]:
f_df.columns

Index(['classes', 'order_approved_at', 'customer_id',
       'customer_zip_code_prefix', 'customer_city', 'customer_state', 'price',
       'freight_value', 'product_id', 'product_weight_g', 'product_length_cm',
       'product_height_cm', 'product_width_cm',
       'product_category_name_english', 'seller_zip_code_prefix',
       'seller_city', 'seller_state', 'order_date', 'order_weekday',
       'order_day', 'order_month', 'customer_seller_state',
       'customer_seller_city', 'customer_seller_zip_code_prefix',
       'product_volume'],
      dtype='object')

## Train, Test 데이터 셋 분리
훈련 및 테스트를 8:2 로 시간에 따라 분리 합니다.

In [13]:

def split_data_2(raw_df, sort_col='order_approved_at',val_ratio=0.3):
    '''
    train, test 데이터 분리
    '''
    df = raw_df.copy()
    val_ratio = 1 - val_ratio # 1 - 0.3  = 0.7

    
    df = df.sort_values(by= sort_col) # 시간 순으로 정렬
    # One-Hot-Encoding
    data1,data2, = np.split(df, 
                     [int(val_ratio * len(df))])   # Randomly sort the data then split out first 70%, second 20%, and last 10%
    
    print(f"data1, data2 shape: {data1.shape},{data2.shape}")
    
    return data1, data2

train_df, test_df = split_data_2(f_df, val_ratio=0.2)




data1, data2 shape: (53740, 25),(13436, 25)


## AutoGluon 용 최종 컬럼 선택

In [14]:
def filter_df(raw_df, cols):
    df = raw_df.copy()
    df = df[cols]
    return df



In [15]:
cols = ['classes', 
       'customer_zip_code_prefix', 'customer_city', 'customer_state', 'price',
       'freight_value', 'product_weight_g', 
       'product_category_name_english', 'seller_zip_code_prefix',
       'seller_city', 'seller_state', 'order_weekday',
       'order_day', 'order_month', 'customer_seller_state',
       'customer_seller_city', 'customer_seller_zip_code_prefix',
       'product_volume']


no_auto_train = filter_df(train_df, cols)
no_auto_test = filter_df(test_df, cols)



## 로컬에 데이터 저장

In [16]:
import os

def save_local(raw_df, preproc_folder, label, file_name):
    df = raw_df.copy()
    df = pd.concat([df[label], df.drop([label], axis=1)], axis=1)
    file_path = os.path.join(preproc_folder, file_name)
    df.to_csv(file_path, index=False)
    print(f'{file_path} is saved')

    
    return file_path

# Train file
preproc_folder = 'preproc_data/auto_no_fe/train'
os.makedirs(preproc_folder, exist_ok=True)    
no_auto_train_file = save_local(no_auto_train, preproc_folder, label='classes', file_name = 'train.csv')

# Test file
preproc_folder = 'preproc_data/auto_no_fe/test'
os.makedirs(preproc_folder, exist_ok=True)    
no_auto_test_file = save_local(no_auto_test, preproc_folder, label='classes',file_name='test.csv' )



preproc_data/auto_no_fe/train/train.csv is saved
preproc_data/auto_no_fe/test/test.csv is saved


In [17]:
%store no_auto_train_file
%store no_auto_test_file

Stored 'no_auto_train_file' (str)
Stored 'no_auto_test_file' (str)
